# Project: Data Modeling with Cassandra

This project is an exercise to perform data modeling with Apache Cassandra, completing an ETL pipeline using Python environment.

The use case is based on a startup called Sparkify that wants to analyze the data they've been collecting on songs and user activity on their new music streaming app. The analysis team is particularly interested in understanding what songs users are listening to.

To complete the project, we will need to model the data by creating tables in Apache Cassandra to run queries. We are provided with part of the ETL pipeline that transfers data from a set of CSV files within a directory to create a streamlined CSV file to model and insert data into Apache Cassandra tables. 

# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [2]:
# Import Python packages 
from prettytable import PrettyTable
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [6]:
# We provide part of the code to set up the CSV file
file = 'event_datafile_new.csv'

# Part II. Complete the Apache Cassandra. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Creating the Cassandra Cluster and setting up the Keyspace environment

#### Creating a Cluster

In [7]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [8]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity_project1 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
try:
    session.set_keyspace('udacity_project1')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data




## 1. Query:
### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### Create Table

In [10]:
query = "CREATE TABLE IF NOT EXISTS song_info_by_session "
query = query + "(session_id int, item_in_session int, artist text, song text, length float, \
                  PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)                              

#### Instert data into tables

In [11]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_session (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Verify with the Query

In [12]:
query = "select artist, song, length from song_info_by_session where session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
x = PrettyTable()
for row in rows:
    x.field_names = ["Artist", "Song", "Length"]
    x.add_row([row.artist, row.song, row.length])
print(x)

+-----------+---------------------------------+--------------------+
|   Artist  |               Song              |       Length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


## 2. Query:
### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### We repeat the process from Query 1

In [13]:
query = "CREATE TABLE IF NOT EXISTS song_info_by_user "
query = query + "(user_id int, session_id int, item_in_session int, artist text, song text, user_name text, \
                  user_surname text, PRIMARY KEY (user_id, session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)              

In [14]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_user (user_id, session_id, item_in_session, artist, song, user_name, \
                  user_surname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
                    

In [15]:
query = "select artist, song, user_name, user_surname from song_info_by_user \
        where user_id = 10 and session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

x = PrettyTable()
for row in rows:
    #print (row.artist, row.song, row.user_name, row.user_surname)
    x.field_names = ["Artist", "Song", "UserName", "UserSurname"]
    x.add_row([row.artist, row.song, row.user_name, row.user_surname])
print(x)

+-------------------+------------------------------------------------------+----------+-------------+
|       Artist      |                         Song                         | UserName | UserSurname |
+-------------------+------------------------------------------------------+----------+-------------+
|  Down To The Bone |                  Keep On Keepin' On                  |  Sylvie  |     Cruz    |
|    Three Drives   |                     Greece 2000                      |  Sylvie  |     Cruz    |
| Sebastien Tellier |                      Kilometer                       |  Sylvie  |     Cruz    |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |  Sylvie  |     Cruz    |
+-------------------+------------------------------------------------------+----------+-------------+


## 3. Query:
### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [16]:
query = "CREATE TABLE IF NOT EXISTS user_name_by_song "
query = query + "(song text, user_id int, user_name text, user_surname text, PRIMARY KEY (song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)  

In [17]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_name_by_song (song, user_id, user_name, user_surname)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [18]:
query = "select user_name, user_surname from user_name_by_song where song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

x = PrettyTable()
for row in rows:
    x.field_names = ["UserName", "UserSurname"]
    x.add_row([row.user_name, row.user_surname])
print(x)

+------------+-------------+
|  UserName  | UserSurname |
+------------+-------------+
| Jacqueline |    Lynch    |
|   Tegan    |    Levine   |
|    Sara    |   Johnson   |
+------------+-------------+


### Drop the tables before closing out the sessions

In [19]:
query = "drop table song_info_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table song_info_by_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table user_name_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()